In [1]:
# 1 Paso - Instalar PySpark en el Contexto de Colab (si es que no está instalado)
# Esto corresponde a línea de comandos (CMD) y no a código Python (!)
!pip install -q pyspark

Creación de Sessión

spark = SparkSession.builder
**Iniciación con constructor**

.appName("ProyectoMetroSantiago")
**Asignar un nombre**

.config("spark.executor.memory", "1g")
**Asignar una cantidad de memoria a cada executor**

.config("spark.driver.memory", "1g")
**Asignar una cantidad de memoria al driver (nodo coordinador)**

.config("spark.sql.shuffle.partitions","2")
**Definir el número de particiones usadas en operaciones como SUFFLE**

.getOrCreate()
**Crear la sessión si es que no existe**

In [2]:
# 2 Paso - Conectarme y crear la sesión en Apache Spark (SparkSession)
from pyspark.sql import SparkSession

# Crear la sesión
spark = SparkSession.builder.appName("ProyectoMetroSantiago").config("spark.executor.memory", "1g").config("spark.driver.memory", "1g").config("spark.sql.shuffle.partitions","2").getOrCreate()

In [3]:
# Cargar el CSV en un dataframe (asegurarse de subir ANTES a la carpeta de recursos el archivo CSV)
df = spark.read.csv("/content/metro_santiago.csv", header=True, inferSchema=True)

# Explorar este dataframe (Apache Spark) con show()
df.show(5)

+---------+-----+-----------+---------------+----------------+-------------------+
|id_evento|línea|   estación| tipo_incidente|duración_minutos|               hora|
+---------+-----+-----------+---------------+----------------+-------------------+
|        1|   L2|   Lo Prado|Falla eléctrica|              15|2025-08-23 11:45:00|
|        2|   L5|Irarrázaval|     Evacuación|              15|2025-08-23 20:00:00|
|        3|   L4|  San Pablo|Falla eléctrica|              25|2025-08-23 19:00:00|
|        4|   L1|   Lo Prado|Falla eléctrica|              30|2025-08-23 05:45:00|
|        5|   L3|Irarrázaval|Falla eléctrica|              20|2025-08-23 08:00:00|
+---------+-----+-----------+---------------+----------------+-------------------+
only showing top 5 rows



In [4]:
# Transformar el DataFrame a RDD
rdd = df.rdd

# Mostrar algunos registros del RDD
rdd.take(5)

[Row(id_evento=1, línea='L2', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 11, 45)),
 Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=3, línea='L4', estación='San Pablo', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 19, 0)),
 Row(id_evento=4, línea='L1', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 5, 45)),
 Row(id_evento=5, línea='L3', estación='Irarrázaval', tipo_incidente='Falla eléctrica', duración_minutos=20, hora=datetime.datetime(2025, 8, 23, 8, 0))]

In [5]:
# Almacenar "in-memory" (en memoria cache) nuestro RDD para utilizarlo varias veces
rdd.cache()

MapPartitionsRDD[18] at javaToPython at NativeMethodAccessorImpl.java:0

In [8]:
# Ver cuántas particiones tiene nuestro RDD
rdd.getNumPartitions()

1

In [10]:
# Reparticionar nuestro RDD, por ejemplo en 4 particiones
rdd = rdd.repartition(4)
rdd.take(5)
# Comparar la creación de los 2 objetos con "supestamente" el mismo HASCODE

In [15]:
# TRANSFORMACIONES - Son operaciones a realizar en el RDD que lo afectan, creando un nuevo objeto RDD
rdd_filtrado = rdd.filter(lambda row: int(row['duración_minutos']) > 20)
rdd_filtrado.take(5)

In [16]:
# TRANSFORMACIÓN - FILTER: Aplica un filtro, o modifica el RDD en base a un criterio lógico
rdd_evacuaciones = rdd.filter(lambda row: row['tipo_incidente'] == "Evacuación")
rdd_evacuaciones.take(5)

[Row(id_evento=34, línea='L4A', estación='Santa Ana', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 9, 15)),
 Row(id_evento=40, línea='L5', estación='Los Dominicos', tipo_incidente='Evacuación', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 18, 45)),
 Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15))]

In [17]:
# TRANSFORMACIÓN - MAP: Función que aplica a cada elemento del RDD y devuelve uno nuevo transformado
rdd_estaciones_mayusculas = rdd.map(lambda row: row['estación'].upper())
rdd_estaciones_mayusculas.take(5)

['FRANKLIN', 'PLAZA EGAÑA', 'CERRILLOS', 'SANTA ANA', 'LOS HÉROES']

In [20]:
# TRANSFORMACIÓN - FLATMAP: Función que, como map(), permite devolver elementos transformados pero varios de una sola entrada
rdd_estaciones = rdd.map(lambda row: row['estación'])
palabras = rdd_estaciones.flatMap(lambda x: x.split(" "))
palabras.take(5)

['Franklin', 'Plaza', 'Egaña', 'Cerrillos', 'Santa']

In [28]:
# TRANSFORMACIÓN - SAMPLE: Permite extraer una muestra aleatoria del RDD
rdd_muestra = rdd.sample(seed=42, fraction=0.3, withReplacement=False)
rdd_muestra.take(5)

[Row(id_evento=31, línea='L4A', estación='Franklin', tipo_incidente='Mantenimiento', duración_minutos=20, hora=datetime.datetime(2025, 8, 23, 6, 15)),
 Row(id_evento=32, línea='L3', estación='Plaza Egaña', tipo_incidente='Falla eléctrica', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 6, 30)),
 Row(id_evento=34, línea='L4A', estación='Santa Ana', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 9, 15)),
 Row(id_evento=35, línea='L5', estación='Los Héroes', tipo_incidente='Retraso', duración_minutos=20, hora=datetime.datetime(2025, 8, 23, 16, 45)),
 Row(id_evento=38, línea='L1', estación='Baquedano', tipo_incidente='Interrupción de servicio', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 22, 45))]

In [29]:
# TRANSFORMACIÓN - UNION: Une 2 RDDs (sin eliminar duplicados)
rdd_duplicado = rdd.filter(lambda row: row['tipo_incidente'] == "Falla eléctrica")
rdd_unido = rdd_evacuaciones.union(rdd_duplicado)
rdd_unido.take(5)

[Row(id_evento=34, línea='L4A', estación='Santa Ana', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 9, 15)),
 Row(id_evento=40, línea='L5', estación='Los Dominicos', tipo_incidente='Evacuación', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 18, 45)),
 Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15))]

In [30]:
# TRANSFORMACIÓN - DISTINCT: Elimina los duplicados (igual que SQL)
rdd_estaciones_distintas = rdd.map(lambda row: row['estación']).distinct()
rdd_estaciones_distintas.take(5)

['Santa Ana', 'Los Dominicos', 'Los Leones', 'Tobalaba', 'Los Héroes']

In [31]:
# TRASFORMACIÓN - SORTBY: Ordena los elementos según un criterio
rdd_ordenado = rdd.sortBy(lambda row: int(row['duración_minutos']))
rdd_ordenado.take(5)

[Row(id_evento=40, línea='L5', estación='Los Dominicos', tipo_incidente='Evacuación', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 18, 45)),
 Row(id_evento=7, línea='L3', estación='Ñuñoa', tipo_incidente='Mantenimiento', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 12, 30)),
 Row(id_evento=13, línea='L5', estación='Los Héroes', tipo_incidente='Falla eléctrica', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 17, 0)),
 Row(id_evento=24, línea='L5', estación='Pajaritos', tipo_incidente='Mantenimiento', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 9, 15)),
 Row(id_evento=25, línea='L2', estación='Tobalaba', tipo_incidente='Falla eléctrica', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 23, 15))]